# Predicting Sentiment Level in Yelp Reviews
## STAT 628 Mon, Group 6, Module 2

## Abstract
Using Rooted Mean Square Error (RMSE) as the performance evalution metric, our score reached 0.58263 on the Kaggle public leaderboard and X.XXX in our own cross-validation (CV) process. We applied NBSVM combined with Xgboost to predict the sentiment level, also demonstrated by the number of stars, in Yelp reviews.

## Data Overview and Cleaning

In [10]:
import pandas as pd
df = pd.read_csv("/Users/yilixia/Downloads/stat628/train_translation.csv")
df.shape

(1546379, 2)

### Translation
More than 99% of the reviews were written in English while the rest of the reviews are in multiple languages including French, Japanese, German, Chinese (Simplified, Traditional and Cantonese). Most of them are well written reviews and contain beneficial information for our future models. Therefore, we decided to translate all the reviews.

In [11]:
# Examples of translated text:
print(df.loc[1489714,'stars'],"-",df.loc[1489714,'text'])
print(df.loc[1286432,'stars'],"-",df.loc[1286432,'text'])

5 - Clean environment .. Address is not very remote. Waiter enthusiasm .... 10-minute drive from the center of Las Vegas.
3 - The interior decoration is very beautiful, things out too slow, the food is very beautiful, but things like the general! Ten dollars on the price a bit too expensive. Will I be too picky!


The information that one review is written in a non-English language wasn't used as a feature label since doing a chi-square test, the results demonstrate that there isn't a significant difference in the distribution of stars between a English review and a non-English review (p-value=0.2202).

### Years
The dates of the Yelp reviews were included into the data, as shown in the plot below, there is a significant trend in the stars over the years, hence we believe that it would be helpful to include them into our model.

(PLOT!)

### And that's all ...
Yes, we are aware of that there are still categories in text data on which we could make efforts. We didn't clean them for the following reasons:

* Uninformative reviews: There are some reviews which contain meaningless text like 'X', 'V', 'O' and 'y' in both the train and test set on which no valid prediction could be made. However, they were automatically exluded during the tokenization process due to `min_df`.
* Hyperlinks: There are two hyperlinks in the text data which were also ignored during the tokenization process.

In [12]:
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
vec = TfidfVectorizer(ngram_range=(1, 2), tokenizer=tokenize,
                      min_df=0.0003, max_df=0.9, strip_accents='unicode', use_idf=1,
                      smooth_idf=1, sublinear_tf=1)

* Stopwords and Snippets dataset: As stated in the paper [Sida&Christopher, 2012](#Sida&Christopher2012), it is not necessary to exclude stopwords and their method performs well on predicting snippet datasets like 'not bad' and 'do not recommend'. Therefore, we did nothing to handle them.
* Mistyped words: This is a category we tried to deal with. However, since we couldn't come up with some general rules to correct mistyped words and manual selection would be exhaustive and trivial in improvement, while in the meantime, existing spell checking methods such as Enchant are not only slow but also imprecise, we didn't do anything.

## Model
### ECOC

Inspired by the trending Kaggle competition on predicting toxic comments ([Kaggle](#Toxic)), we redefined the problem to a binary one vs all problem by asking the question *"Is the review a 1/2/3/4/5 star review?"* or *"What is the probability that the review is a 1/2/3/4/5 star review?"* five times instead of predicting the stars of the reviews in one run. This process is also know as Error-Correcting Output Coding (ECOC) introduced by Dietterich et al ([Dietterich et al, 1991](#Dietterich1991))([Dietterich&Bakiri, 1995](#Dietterich1995)) and is used for improving multiclass learning problems by seperating multiple classes using the Hamming distance.

<img src="../Plots/ECOC_example.jpg"/> #SRC DOESNT WORK CORRECTLY?

There are many advantages under the new framework. Besides the advantages that were already introduced in the paper, such as that there are many available methods which do binary classification or predicting problems better and faster than multiclass classification problems, applying ECOC also provides a natural smoothing procedure by calculating the predicted stars using the mean or expectation to a real value instead of an integer. Hence it would more likely to decrease the RMSE. 

Of course, Dietterich also pointed out that ECOC comes with some additional costs ([Dietterich&Bakiri, 1995](#Dietterich1995)). It would increase the complexity of the model as well as the training time. However, in our case, we were looking for better prediction performance thus a complicated model would only be of our least concerns. Furthermore, the train-predict time was actually incredibly fast. It would usually take a little less than 3 minutes (170s) to train on 200,000 and predict on 200,000 samples using NB/SVM which we'll introduce in the section below.

### NBSVM
NBSVM, short for Naive Bayes Support Vector Machines (NBSVM), was introduced by Sida and Christopher ([Sida&Christopher, 2012](#Sida&Christopher2012)) in 2012 and is a prevailing method in Kaggle competitions. It had been confirmed by Rennie et al ([Rennie et al, 2001](#Rennie2001)) that the Support Vector Machine (SVM) consistently outperforms the Naive Bayes (NB) while both of them are popular sentiment classifying methods in previous researches. Though this finding is consistent with our experience on the Yelp sentiment prediction, SVM alone was never enough to cross the pre-set borderline (RMSE=0.71). In his paper, Sida stated that NB is better at snippets while SVM is better at full-length reviews. By combining the advantages of NB and SVM, NBSVM either exceeds or approaches most of the previous methods including those that use sentiment lexicons and hard-coded reversal rules.

Combining ECOC and NBSVM ([NBSVM-Code](#Code)), we estimated the probability of whether the review is a 1/2/3/4/5 star review. Afterwards, we used the expectation, which was calculated by multiplying the number of stars with the normalized probabilities, as our predicted sentiment level and reached a Kaggle public leaderboard score of 0.68886 and a RMSE of about 0.63 using CV with 200,000 samples as training set and 200,000 samples as testing set.

### Xgboost

## Ensemble
Empirically, ensembling results of different methods where the predictions are relatively accurate and uncorrelated will improve the prediction performance.

(Make examples misclassified by current classifier more important in the next classifier
(Create classifiers using different training sets, where each training set is created by “bootstrapping,” i.e., drawing examples (with replacement) from all possible training examples

## Conclusion

## Reference
1. <a id='Toxic'></a> https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge
2. <a id='Dietterich1991'></a> Dietterich, Thomas G., and Ghulum Bakiri. "Error-correcting output codes: A general method for improving multiclass inductive learning programs." AAAI. 1991.
3. <a id='Dietterich1995'></a> Dietterich, Thomas G., and Ghulum Bakiri. "Solving multiclass learning problems via error-correcting output codes." Journal of artificial intelligence research 2 (1995): 263-286.
4. <a id='Sida&Christopher2012'></a> Wang, Sida, and Christopher D. Manning. "Baselines and bigrams: Simple, good sentiment and topic classification." Proceedings of the 50th Annual Meeting of the Association for Computational Linguistics: Short Papers-Volume 2. Association for Computational Linguistics, 2012.
5. <a id='Rennie2001'></a> Rennie, Jason DM, and Ryan Rifkin. "Improving multiclass text classification with the support vector machine." (2001).
6. <a id='Code'></a>https://github.com/sidaw/nbsvm; https://github.com/mesnilgr/nbsvm; https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline